In [13]:
import os
import sys
import pandas as pd
import numpy as np
from neuroCombat import neuroCombat

try:
    # Works if running from a Python script
    current_file = os.path.abspath(__file__)
except NameError:
    # Fallback for Jupyter Notebook or IPython
    current_file = os.path.abspath('')

project_dir = os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(current_file))) ,'germany/FL/Experiments_1_2_eNKI/silo_Datasets')
sys.path.append(project_dir)


In [14]:
silo = 'CamCAN' #or 'SALD'
silo_path = os.path.join(project_dir,f'{silo}/Train_{silo}.csv')
central_path = os.path.join(project_dir,f'eNKI/Train_eNKI.csv')

print(f"Loading data from {silo_path} and {central_path}")

Loading data from /home/tanurima/germany/FL/Experiments_1_2_eNKI/silo_Datasets/CamCAN/Train_CamCAN.csv and /home/tanurima/germany/FL/Experiments_1_2_eNKI/silo_Datasets/eNKI/Train_eNKI.csv


In [15]:
# Example: Load data from two scanners
# Replace with your actual data paths
df1 = pd.read_csv(silo_path)
df2 = pd.read_csv(central_path)  # Shape: (features, subjects)

# Get feature columns (assuming columns are named '1', '2', ..., '1073')
feature_columns = [str(i) for i in range(1, 1074)]  # Generates ['1', '2', ..., '1073']
#print('feature columns',feature_columns)
# Convert to numpy arrays
X_1 = np.transpose(df1[feature_columns].to_numpy()).astype(np.float32)  #Features matrix
y_1 = np.transpose(df1['age'].to_numpy()).astype(np.float32).tolist()            #Target vector

# Convert to numpy arrays
X_2 = np.transpose(df2[feature_columns].to_numpy()).astype(np.float32)  #Features matrix
y_2 = np.transpose(df2['age'].to_numpy()).astype(np.float32).tolist()            #Target vector

# Combine datasets horizontally (columns = all subjects from both scanners)
data_combined = np.hstack([X_1, X_2])  # Shape: (features, subjects_total)
age_list = y_1 + y_2  # Shape: (subjects_total,)

In [16]:
# Example: 5 subjects from scanner1 and 5 from scanner2
n_scanner1 = X_1.shape[1]  # Number of subjects in scanner1
n_scanner2 = X_2.shape[1]  # Number of subjects in scanner2

covars = {
    'batch': [1]*n_scanner1 + [2]*n_scanner2,  # Scanner IDs
    'age':  age_list  # Example continuous covariate [30,25,40,35,28, 32,45,50,38,42]
}
covars = pd.DataFrame(covars)

In [17]:
# Harmonize the data
harmonized_output = neuroCombat(
    dat=data_combined,
    covars=covars,
    batch_col='batch',              # Column name for scanner IDs
    categorical_cols=[],    # Specify categorical variables
    # Optional parameters:
    eb=True,             # Use Empirical Bayes (recommended)
    parametric=True,      # Parametric adjustment (default)
    mean_only=False,      # Adjust both mean and variance (default)
    ref_batch=None        # Harmonize to overall average (set to 1 or 2 to use a scanner as reference)
)

# Extract harmonized data
data_harmonized = harmonized_output["data"]
data_harmonized.shape


[neuroCombat] Creating design matrix
[neuroCombat] Standardizing data across features
[neuroCombat] Fitting L/S model and finding priors
[neuroCombat] Finding parametric adjustments
[neuroCombat] Final adjustment of data


(1073, 1315)

In [18]:
# Split harmonized data back into original scanners (if needed)
harmonized_scanner1 = data_harmonized[:, :n_scanner1]
harmonized_scanner2 = data_harmonized[:, n_scanner1:]

y_1 = np.transpose(df1['age'].to_numpy()).astype(np.float32).tolist()            #Target vector
y_2 = np.transpose(df2['age'].to_numpy()).astype(np.float32).tolist()            #Target vector

# Move this line before creating harmonized_combined_data
feature_columns.append('age')
print('feature columns:',feature_columns)

print("Harmonized data for scanner1:")
print(harmonized_scanner1.T.shape)
print(  "Target variable for scanner1:")
age = np.array(y_1).reshape(-1, 1)
print(age.shape)

# Concatenate the arrays horizontally (adding target as an extra column)
harmonized_combined_data = np.concatenate((harmonized_scanner1.T, age), axis=1)
print('harmonized data shape:',harmonized_combined_data.shape)
harmonized_train = pd.DataFrame(harmonized_combined_data, columns=feature_columns)
harmonized_path = silo_path.replace('Train','Train_harmonized')
harmonized_train.to_csv(harmonized_path)


feature columns: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156'

In [19]:
def apply_combat_harmonization(test_path, combat_params, batch_col='batch', categorical_cols=[]):
    """
    Apply pre-trained Combat harmonization to new data.

    Args:
        test_path (str): Path to new test data CSV
        combat_params (dict): Saved Combat parameters from training, containing:
                              - "estimates": Model parameters
                              - "batch_col": Batch column name (e.g., "batch")
                              - "categorical_cols": List of categorical covariates
    """
    # Load test data
    test_data = pd.read_csv(test_path)

    # Clean data (drop rows with missing age)
    test_data_clean = test_data.dropna(subset=['age'], axis=0)

    # Check if test data is empty after cleaning
    if test_data_clean.empty:
        raise ValueError("Test data is empty after dropping rows with missing 'age'.")

    # 1. Add batch column to test_data_clean
    # Assign all test data to a single batch (e.g., batch=1)
    test_data_clean['batch'] = 1  # Replace 1 with your desired batch label

    # 2. Extract covariates (MUST include 'batch' and other training covariates)
    required_covars = ['age', 'batch']  # Add other covariates used during training
    covars_test = test_data_clean[required_covars]

    # 3. Extract features (columns '1' to '1073')
    new_data = test_data_clean.loc[:, '1':'1073']

    # Transpose to (features x subjects) format
    new_data_t = new_data.T

    # 4. Apply pre-trained Combat parameters
    # Instead of passing estimates directly to neuroCombat,
    # you would need to modify the input data (new_data_t)
    # based on combat_params["estimates"] before calling neuroCombat.
    # For instance, you might apply the estimated batch effects
    # to new_data_t using operations like addition, subtraction,
    # multiplication, or division, as appropriate based on the
    # specific Combat model you are using.

    # Example: Let's assume you need to add a batch effect
    # This is a simplified example, you need to adapt it based on your
    # actual model and the content of combat_params["estimates"]
    # batch_effect = combat_params["estimates"]["batch_effect"]  # Replace with actual key
    # new_data_t = new_data_t + batch_effect

    # Since neuroCombat doesn't accept 'estimates', remove it from the call:
    harmonized = neuroCombat(
        dat=new_data_t,
        covars=covars_test,
        batch_col=batch_col,
        categorical_cols=categorical_cols,
        # estimates=combat_params["estimates"]  # Remove this line
    )["data"]

    # Convert back to DataFrame and add age labels
    harmonized_df = pd.DataFrame(harmonized.T, columns=new_data.columns)
    harmonized_df['age'] = test_data_clean['age'].values

    return harmonized_df


In [20]:
# Call apply_combat_harmonization()
#test_path = '/content/CamCAN_test.csv'  # Path to your test data CSV
# After initial harmonization
combat_params = {
    "estimates": harmonized_output["estimates"],
    "batch_col": "batch",
    "categorical_cols": []
}
silo_test_path = silo_path.replace('Train','Test')
harmonized_test_df = apply_combat_harmonization(
    test_path=silo_test_path,
    combat_params=combat_params,
    batch_col='batch',
    categorical_cols=[]
)
harmonized_test_path = silo_test_path.replace('Test','Test_harmonized')
harmonized_test_df.to_csv(harmonized_test_path)

[neuroCombat] Creating design matrix
[neuroCombat] Standardizing data across features
[neuroCombat] Fitting L/S model and finding priors
[neuroCombat] Finding parametric adjustments
[neuroCombat] Final adjustment of data
